In [1]:
import cv2

In [5]:
import cv2 

# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = {
                "손목": 0,
                "엄지0": 1, "엄지1": 2, "엄지2": 3, "엄지3": 4,
                "검지0": 5, "검지1": 6, "검지2": 7, "검지3": 8,
                "중지0": 9, "중지1": 10, "중지2": 11, "중지3": 12,
                "약지0": 13, "약지1": 14, "약지2": 15, "약지3": 16,
                "소지0": 17, "소지1": 18, "소지2": 19, "소지3": 20,
            }

POSE_PAIRS = [["손목", "엄지0"], ["엄지0", "엄지1"],
                ["엄지1", "엄지2"], ["엄지2", "엄지3"],
                ["손목", "검지0"], ["검지0", "검지1"],
                ["검지1", "검지2"], ["검지2", "검지3"],
                ["손목", "중지0"], ["중지0", "중지1"],
                ["중지1", "중지2"], ["중지2", "중지3"],
                ["손목", "약지0"], ["약지0", "약지1"],
                ["약지1", "약지2"], ["약지2", "약지3"],
                ["손목", "소지0"], ["소지0", "소지1"],
                ["소지1", "소지2"], ["소지2", "소지3"]]
# 각 파일 path
protoFile = "pose_deploy.prototxt"
weightsFile = "pose_iter_102000.caffemodel"
 
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

# 이미지 읽어오기
image = cv2.imread("./img/51.jpg")

# frame.shape = 불러온 이미지에서 height, width, color 받아옴
imageHeight, imageWidth, _ = image.shape
 
# network에 넣기위해 전처리
inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)
 
# network에 넣어주기
net.setInput(inpBlob)

# 결과 받아오기
output = net.forward()

# output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
H = output.shape[2]
W = output.shape[3]
print("이미지 ID : ", len(output[0]), ", H : ", output.shape[2], ", W : ",output.shape[3]) # 이미지 ID

# 키포인트 검출시 이미지에 그려줌
points = []
for i in range(0,21):
    # 해당 신체부위 신뢰도 얻음.
    probMap = output[0, i, :, :]

    # global 최대값 찾기
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

    # 원래 이미지에 맞게 점 위치 변경
    x = (imageWidth * point[0]) / W
    y = (imageHeight * point[1]) / H

    # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
    if prob > 0.1 :    
        cv2.circle(image, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)       # circle(그릴곳, 원의 중심, 반지름, 색)
        cv2.putText(image, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
        points.append((int(x), int(y)))
    else :
        points.append(None)

cv2.imshow("Output-Keypoints",image)
cv2.waitKey(0)

# 이미지 복사
imageCopy = image

# 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
for pair in POSE_PAIRS:
    partA = pair[0]             # Head
    partA = BODY_PARTS[partA]   # 0
    partB = pair[1]             # Neck
    partB = BODY_PARTS[partB]   # 1
    
    #print(partA," 와 ", partB, " 연결\n")
    if points[partA] and points[partB]:
        cv2.line(imageCopy, points[partA], points[partB], (0, 255, 0), 2)


cv2.imshow("Output-Keypoints",imageCopy)
cv2.waitKey(0)
cv2.destroyAllWindows()

이미지 ID :  22 , H :  43 , W :  43
